# 案例一（自定义某层的计算逻辑）

In [1]:
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.layers import *
from tensorflow.keras.utils import plot_model, CustomObjectScope
from tensorflow.keras import utils
from tensorflow.keras.datasets import cifar10, mnist
from tensorflow.keras.models import load_model, save_model, Sequential, Model
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow import keras
import numpy as np

import os

## 定义层

In [2]:
class CustomLayer(layers.Layer):
    def __init__(self, units=32, **kwargs):              # init函数是用来设定默认值（推荐）
        self.units = units                               # 也可以在init函数中对变量初始化，但不推荐
        super(CustomLayer, self).__init__(**kwargs)      # **kwargs可以避免name问题
        
    def build(self, input_shape):                        # build函数是用来初始化变量（推荐）
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='random_uniform',
            trainable=True,
            name='w'
        )
        self.b = self.add_weight(
            shape=(self.units, ),
            initializer='random_uniform',
            trainable=True,
            name='b'
        )
        
    def call(self, inputs):                               # call函数是使用逻辑（在底层自动条用build）
        return tf.matmul(inputs, self.w) + self.b
    
    def get_config(self):                                 # 为了让模型能够保存，需要重写get_config()
        config = super(CustomLayer, self).get_config()
        config.update({"units":self.units})
        return config

## 定义激活函数（可选）

In [4]:
def myActivation(x):
    return tf.nn.tanh(x) ** 2

## 搭建模型

In [5]:
input_layer = Input(shape=(32, ))
middle_layer = CustomLayer(10)(input_layer)
act_layer = Activation(myActivation)(middle_layer)
output_layer = Dense(10)(middle_layer)

model = Model(inputs=input_layer, outputs=output_layer)

## 保存模型

In [6]:
model.save('temp.h5')

## 从文件中载入模型

### 方法一

In [7]:
new_model = load_model('temp.h5', custom_objects={"CustomLayer":CustomLayer, "myActivation":myActivation})

### 方法二

In [8]:
with CustomObjectScope({'CustomLayer':CustomLayer, "myActivation":myActivation}):
    model = load_model('temp.h5')